In [ ]:
from Bio import AlignIO
from collections import Counter

def extract_conserved_regions(input_file, output_file, threshold=0.9):
    # Carregar o alinhamento
    alignment = AlignIO.read(input_file, "fasta")
    num_sequences = len(alignment)

    # Verificar se o número de sequências é suficiente
    if num_sequences == 0:
        print("⚠️ Nenhuma sequência encontrada no alinhamento.")
        return

    # Identificar regiões conservadas conforme o threshold
    output_sequence = []
    conserved_count = 0
    conserved_info = []  # Lista para armazenar informações sobre as regiões conservadas

    for i in range(len(alignment[0])):  # Para cada posição na sequência alinhada
        column = [record.seq[i] for record in alignment]  
        count_residues = Counter(column)

        if '-' in count_residues:
            del count_residues['-']  # Remover gaps para não contá-los

        if count_residues:
            most_common_residue, most_common_count = count_residues.most_common(1)[0]  
            conservation_ratio = most_common_count / num_sequences  

            if conservation_ratio >= threshold:
                output_sequence.append(most_common_residue)  # Região conservada
                conserved_count += 1
                conserved_info.append((i, conservation_ratio * 100, most_common_residue))  # Adiciona à lista
            else:
                output_sequence.append('-')  # Não conservado
        else:
            output_sequence.append('-')  # Apenas gaps na posição

    # Salvar as regiões conservadas em um arquivo
    with open(output_file, "w") as output:
        output.write(f">Conserved_Region   Conserved_Residues: {conserved_count} / {len(alignment[0])}   Threshold: {threshold*100}%\n")
        output.write("".join(output_sequence) + "\n")

    print(f"✅ Arquivo '{output_file}' gerado com threshold de {threshold*100}%.")
    
    # Imprimir a lista de regiões conservadas (posição, porcentagem de conservação, aminoácido)
    print(f"\nRegiões conservadas (posição, porcentagem de conservação, aminoácido):")
    for pos, perc, aa in conserved_info:
        print(f"Posição: {pos + 1}, Porcentagem: {perc:.2f}%, Aminoácido: {aa}")

# Exemplo de uso
extract_conserved_regions("gagprotein.fasta", "gagconserved.fasta", threshold=0.90)
